In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
inflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-inflows.pqt')
outflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-outflows.pqt')

# remove rows where memo = category
outflow = outflow[outflow['memo'] != outflow['category']]

outflow_ids = set(outflow["prism_consumer_id"].unique())
inflow_ids = set(inflow["prism_consumer_id"].unique())

in_not_out = inflow_ids - outflow_ids
out_not_in = outflow_ids - inflow_ids

#consumers in both inflow and outflow
consumers_both = sorted(set(inflow["prism_consumer_id"]).intersection(outflow["prism_consumer_id"]))

#80-20 train test split
train_ids, test_ids = train_test_split(consumers_both, test_size=0.2, random_state=42)

inflow_train = inflow[inflow["prism_consumer_id"].isin(train_ids)]
inflow_test  = inflow[inflow["prism_consumer_id"].isin(test_ids)]

outflow_train = outflow[outflow["prism_consumer_id"].isin(train_ids)]
outflow_test  = outflow[outflow["prism_consumer_id"].isin(test_ids)]

# print(f'Inflow_train: {inflow_train["amount"].median()}\nInflow_test: {inflow_test["amount"].median()}\nOutflow_train: {outflow_train["amount"].median()}\nOutflow_test: {outflow_test["amount"].median()}')

In [3]:
inflow_train

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
0,0,acc_0,PAYCHECK,2477.02,2022-03-18,PAYCHECK
1,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-10-25,EXTERNAL_TRANSFER
2,0,acc_0,MISCELLANEOUS,6.29,2022-08-26,MISCELLANEOUS
3,0,acc_0,EXTERNAL_TRANSFER,277.00,2022-06-03,EXTERNAL_TRANSFER
4,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-07-29,EXTERNAL_TRANSFER
...,...,...,...,...,...,...
512373,5939,acc_9522,DEPOSIT,512.00,2022-12-05,DEPOSIT
512374,5939,acc_9522,REFUND,107.35,2022-12-13,REFUND
512375,5939,acc_9522,DEPOSIT,576.00,2023-01-03,DEPOSIT
512376,5939,acc_9522,DEPOSIT,95.34,2023-01-18,DEPOSIT


In [4]:
inflow_train['category'].value_counts()

category
EXTERNAL_TRANSFER        123425
SELF_TRANSFER             84520
DEPOSIT                   48246
MISCELLANEOUS             44985
PAYCHECK                  25783
PAYCHECK_PLACEHOLDER      20559
REFUND                    18292
INVESTMENT_INCOME         13430
SMALL_DOLLAR_ADVANCE      10936
OTHER_BENEFITS             6073
TAX                        2688
LOAN                       1932
UNEMPLOYMENT_BENEFITS      1476
INSURANCE                   156
Name: count, dtype: int64

In [5]:
inflow

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
0,0,acc_0,PAYCHECK,2477.02,2022-03-18,PAYCHECK
1,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-10-25,EXTERNAL_TRANSFER
2,0,acc_0,MISCELLANEOUS,6.29,2022-08-26,MISCELLANEOUS
3,0,acc_0,EXTERNAL_TRANSFER,277.00,2022-06-03,EXTERNAL_TRANSFER
4,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-07-29,EXTERNAL_TRANSFER
...,...,...,...,...,...,...
513110,5941,acc_9524,EXTERNAL_TRANSFER,8.66,2023-01-21,EXTERNAL_TRANSFER
513111,5941,acc_9524,EXTERNAL_TRANSFER,267.13,2023-01-23,EXTERNAL_TRANSFER
513112,5941,acc_9524,EXTERNAL_TRANSFER,2.00,2023-01-24,EXTERNAL_TRANSFER
513113,5941,acc_9524,EXTERNAL_TRANSFER,207.16,2023-01-24,EXTERNAL_TRANSFER


1. sort dates per consumer
2. time difference btwn transaction
3. grouping similar amounts --> with 10% difference (within same account)
4. test if recurring on weekly/biweekly/monthly

Assumptions:
- paychecks given on a timely basis
- paycheck amounts are within a 10% range
- paychecks are deposited into the same account (???)

In [6]:
# inflow_train sorted
# inflow_train.loc[:,'posted_date'] = pd.to_datetime(inflow_train['posted_date'])
inflow_train['posted_date'] = pd.to_datetime(inflow_train['posted_date'], errors='coerce')
it_sorted = inflow_train.sort_values(['prism_consumer_id', 'posted_date'])
it_sorted['interval'] = it_sorted.groupby('prism_consumer_id')['posted_date'].diff().dt.days
it_sorted['is_recurring_income'] = False

for cid, temp in it_sorted.groupby('prism_consumer_id'):
        
    # Extract amounts
    amounts = temp['amount'].values
    
    # Median-based amount cluster detection
    # Wage Growth in the United States averaged 6.16 percent from 1960 until 2025 (from us Bereau of economic analysis)
    # https://tradingeconomics.com/united-states/wage-growth#:~:text=Wage%20Growth%20in%20the%20United,U.S.%20Bureau%20of%20Economic%20Analysis
    median_amt = np.median(amounts)
    amount_mask = np.abs(amounts - median_amt) < 0.10 * median_amt  # ±10%
    
    # # If fewer than 3 similar amounts → cannot be recurring
    # if amount_mask.sum() < 3:
    #     continue
        
    # Check timing pattern ONLY among similar-amount inflows
    similar_dates = temp.loc[amount_mask, 'posted_date']
    intervals = similar_dates.diff().dt.days.dropna()
    
    if len(intervals) < 2:
        continue
    
    interval_mean = intervals.mean()
    interval_std  = intervals.std()
    
    # Timing rule: weekly, biweekly, monthly patterns
    recurring = False
    
    # weekly (~7 days)
    if 6 <= interval_mean <= 8 and interval_std < 3:
        recurring = True
        
    # biweekly (~14 days)
    if 13 <= interval_mean <= 16 and interval_std < 4:
        recurring = True
        
    # monthly (~30 days)
    if 27 <= interval_mean <= 33 and interval_std < 8:
        recurring = True
    
    # Mark as recurring for this customer
    if recurring:
        it_sorted.loc[temp.index[amount_mask], 'is_recurring_income'] = True
    
it_sorted

/tmp/ipykernel_892/3354073408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inflow_train['posted_date'] = pd.to_datetime(inflow_train['posted_date'], errors='coerce')


,prism_consumer_id,prism_account_id,memo,amount,posted_date,category,interval,is_recurring_income
73,0,acc_0,EXTERNAL_TRANSFER,37000.00,2022-02-14,EXTERNAL_TRANSFER,NaN,False
14,0,acc_1,INVESTMENT_INCOME,0.04,2022-02-17,INVESTMENT_INCOME,3.0,False
18,0,acc_0,EXTERNAL_TRANSFER,25.00,2022-02-18,EXTERNAL_TRANSFER,1.0,False
27,0,acc_0,PAYCHECK,2331.71,2022-02-18,PAYCHECK,0.0,False
20,0,acc_0,EXTERNAL_TRANSFER,75.00,2022-02-24,EXTERNAL_TRANSFER,6.0,False
...,...,...,...,...,...,...,...,...
512373,5939,acc_9522,DEPOSIT,512.00,2022-12-05,DEPOSIT,10.0,False
512374,5939,acc_9522,REFUND,107.35,2022-12-13,REFUND,8.0,False
512375,5939,acc_9522,DEPOSIT,576.00,2023-01-03,DEPOSIT,21.0,False
512376,5939,acc_9522,DEPOSIT,95.34,2023-01-18,DEPOSIT,15.0,False


In [7]:
it_sorted['is_recurring_income'].value_counts()

is_recurring_income
False    400265
True       2236
Name: count, dtype: int64

In [8]:
it_sorted[it_sorted['is_recurring_income'] == True]['category'].value_counts()

category
PAYCHECK_PLACEHOLDER     430
PAYCHECK                 369
SELF_TRANSFER            307
OTHER_BENEFITS           295
DEPOSIT                  280
EXTERNAL_TRANSFER        239
MISCELLANEOUS            235
UNEMPLOYMENT_BENEFITS     59
SMALL_DOLLAR_ADVANCE      15
REFUND                     4
TAX                        3
Name: count, dtype: int64

In [16]:
it_sorted[it_sorted['is_recurring_income'] == True].sample(10)

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category,interval,is_recurring_income
11761,132,acc_365,SELF_TRANSFER,50.00,2022-03-11,SELF_TRANSFER,1.0,True
36410,394,acc_1169,PAYCHECK,2437.56,2020-07-17,PAYCHECK,2.0,True
255052,2941,acc_6524,EXTERNAL_TRANSFER,50.00,2023-06-16,EXTERNAL_TRANSFER,1.0,True
474661,5506,acc_9089,DEPOSIT,220.00,2021-04-30,DEPOSIT,3.0,True
498907,5796,acc_9379,DEPOSIT,1470.00,2022-06-21,DEPOSIT,7.0,True
86475,920,acc_2638,PAYCHECK,814.24,2022-06-01,PAYCHECK,1.0,True
86671,928,acc_2654,OTHER_BENEFITS,1325.31,2022-06-09,OTHER_BENEFITS,9.0,True
106286,1131,acc_3207,SELF_TRANSFER,300.00,2021-12-31,SELF_TRANSFER,0.0,True
422500,4891,acc_8474,PAYCHECK_PLACEHOLDER,1026.22,2021-12-01,PAYCHECK_PLACEHOLDER,5.0,True
176136,1919,acc_5333,PAYCHECK_PLACEHOLDER,471.01,2022-01-14,PAYCHECK_PLACEHOLDER,7.0,True
